### 리뷰 기반 언어사전 구축 및 사용자 정보 추출(11 관측치 처리 포함)

필요 라이브러리 로드

In [1]:
import pandas as pd;import numpy as np;import re, os, sys, json
from pykospacing import Spacing #띄어쓰기
spacing=Spacing()
from konlpy.tag import Kkma #형태소분석
from konlpy.tag import Okt #형태소분석
import soynlp; from soynlp.normalizer import *
from konlpy.tag import Komoran, Hannanum, Kkma, Okt
#komoran = Komoran(); hannanum = Hannanum(); kkma = Kkma(); okt = Okt()
import time

함수 정의

In [7]:
#(1) 크롤링 데이터 병합
def crawlingdataconcat():
    df = pd.DataFrame()
    for i in range(1,21,1):
        try:
            df_before = pd.read_csv('data/hood_'+str(i)+'page.csv') #크롤링파일 불러오기
            print('df',str(i),': ',df_before.shape,end = ',')
        except:
            df_before = pd.DataFrame() #아직크롤링되지 않은파일은 빈 df
            print('df',str(i),': ',df_before.shape)
            
        df = pd.concat([df,df_before]) #합치기
            

            
    print('모두합친df : ', df.shape)
    df.drop_duplicates(subset = None,keep = 'first', inplace = True,ignore_index = True)
    print('중복제거df : ', df.shape)#전체열이 같은 중복제거
    #성별, 키,몸무게 모두 null아니고 수치종류 적어도 하나 값이 있는 것들 filter
    df = df[~df['gender'].isnull()&~df['height'].isnull()&~df['weight'].isnull()&
                                      (~df['총장'].isnull()|
                                       ~df['어깨너비'].isnull()|~df['가슴단면'].isnull()|
                                       ~df['소매길이'].isnull())]
    print('최종사용 data:{}'.format(df.shape))
    print('-'*10)
    return df

#(2) 크롤링 데이터 기초전처리
def crawlingdataprocessing(df):
    #kg,cm제거 및 수치타입으로 변경
    df["height"]=df["height"].replace('cm','',regex = True)
    df["weight"]=df["weight"].replace('kg','',regex = True)
    df= df.astype({'height':'float','weight':'float'})
    print('df전처리완료')
    print('-'*10)
    return df

#(3) 단순자연어전처리 컬럼 review 생성
def add_reviewcol(df):
    df = df.drop_duplicates(subset=['content'])#리뷰중복제거
    print('같은내용리뷰 중복제거 완료')
    df.reset_index(drop=True, inplace=True)

    df['review'] = str('')
    #외국어 리뷰 삭제
    i = 0; nohangul = []
    for i in range(df.shape[0]):
        text = re.sub('[^ㄱ-ㅣ가-힣]', '',df.iloc[i,8])
        if(text==''):
            nohangul.append(i)
    df = df.iloc[[True if i not in nohangul else False for i in range(df.shape[0])],:]
    df.reset_index(drop=True, inplace=True)

    i=0
    for i in range(df.shape[0]):
        text = df.iloc[i,7]
        text = re.sub(pattern='[^\w\s\n]', repl='', string=text) #특수문자 제거
        text = re.sub(pattern='[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z]', repl=' ', string=text) #숫자, 그이외 삭제
        text = re.sub(pattern='[ㄱ-ㅎㅏ-ㅣ]+', repl='', string=text) #단순 모음, 자음 삭제
        text = repeat_normalize(text, num_repeats=2) #불필요반복문자정규화
        text = spacing(text) #띄어쓰기
        df['review'][i] = text
    print('리뷰 전처리 열 추가')
    print('df shape:{}'.format(df.shape))
    print('-'*10)
    return df

#(4) 사이즈 단어사전 중복 탐색 함수
# 두개의 사전이 합치고 나서 진행
# 중복제거
# 'n+2'개의 최종 리스트가 생성됨
def sizedict_nodup(oneOfRawList):
    print('중복제거 전:{}개'.format(len(oneOfRawList)))
    oneOfRawList = set(oneOfRawList)
    print('중복제거 후:{}개'.format(len(oneOfRawList)))
    print('-'*10)
    return list(oneOfRawList)

사용 데이터 불러오기

In [3]:
#자연어전처리 완료된 review 컬럼 추가 전 파일
df=pd.read_csv('data/hood_before_add_review.csv', encoding='utf-8')
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22502 entries, 0 to 22501
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user         22502 non-null  object 
 1   gender       22502 non-null  object 
 2   height       22502 non-null  float64
 3   weight       22502 non-null  float64
 4   item         22502 non-null  object 
 5   size         22502 non-null  object 
 6   star         22502 non-null  float64
 7   content      22502 non-null  object 
 8   size_eval    22502 non-null  object 
 9   bright_eval  22502 non-null  object 
 10  color_eval   22502 non-null  object 
 11  thick_eval   22502 non-null  object 
 12  cm           22502 non-null  object 
 13  총장           22502 non-null  float64
 14  어깨너비         21561 non-null  float64
 15  가슴단면         22502 non-null  float64
 16  소매길이         22385 non-null  float64
dtypes: float64(7), object(10)
memory usage: 2.9+ MB


(22502, 17)

In [59]:
#자연어 전처리된 review 컬럼 생성
start = time.time()
df_before_nlp = add_reviewcol(df)
end = time.time()
print("{:.5f} sec".format(end-start))
df_before_nlp_origin = df_before_nlp.copy()

같은내용리뷰 중복제거 완료


<ipython-input-55-4572b4dce6c7>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = str('')


1/1 [==============================] - 0s 32ms/step


<ipython-input-55-4572b4dce6c7>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'][i] = text


1/1 [==============================] - 0s 29ms/step


...

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 29ms/step
리뷰 전처리 열 추가
df shape:(20037, 18)
----------
1579.76167 sec


In [64]:
#spacing 시간 오래걸려서 파일 저장 후 빼서 사용
df_before_nlp.to_pickle('data/df_before_nlp.pickle')

In [4]:
df_before_nlp = pd.read_pickle('data/df_before_nlp.pickle')

사이즈 언어사전 정의

In [5]:
total = ['사이즈','사이즈하','사이즈감','임사이즈','여사이즈','사이즌','뻐요사이즈','싸이즈','핏이','핏','핏입니다',
      '핏감도','핏나','핏감이랑','핏처럼','핏입니당','핏이에요','핏이예요','핏이랑','핏이구요','핏이네요','핏감은',
      '핏입니','핏이라서','핏으로','옷핏','핏더','핏도','핏감','옷','후드','후드티','핏은','좋아요핏','요핏도',
      '좋아요핏도','상의','요핏','폼','폭','사이즈','후드','옷','품','전체','품도','몸통','옷핏','핏',
      '핏이구','핏이었','핏임','핏입',
      '핏입니','핏감','요핏','폼','후드핏','핏이','크기','상품','상의','핏더']
chongjang = ['길이','기장','총장','궁디','궁딩','엉덩이','밑','밑단','위아래','끝단','밑기장',
             '밑단이랑','기장이','기장입니','밑단','밑','기장감','요기장','총길이','총장','총기장',
            '길이감','끝','기장','길이']
shoulder = ['어깨핏','어께','어깨','너비','어깡','골격','바디','가로','넓이','등판','통',
            '어깨깡패','어깨라','어깨넓','어깨쪽','어께','몸집','어깨핏','어깨선','어깨라인','어깡',
            '어깨','통','몸통','골격','넓이','너비','등판']

chest = ['가슴','둘레','바디','가로','넓이','통','둘레','몸집','가슴팍','통','몸통']
arm = ['소매','팔도','손목','손','팔다리','팔목','팔','팔꿈치','팔이','요팔',
       '당팔','팔길이','팔다리','팔길','팔소매','손목','팔목','소매통','손','팔','소매','팔기장']

small = ['작습니다','짧았어요','짧습니다','짧으면','짧','작았어요','짧다고','쪼여서','작았으면',
      '작아도','작으면','작음','작아진','좁다고',
      '짧아요','짧게','짧다는','좁다는','작네','작기는','달라붙는','작은데','짧은데','짧음',
      '작긴','짧네요','작게','작아서','작은','짧고','짧아','좁은','타이트','짧았지만','붙어서','짧긴',
      '작은것','작아요','짧은','짧아서','짧지만','좁아','좁고','작네요','작아','쪼이는','좁아요','크롭느낌',
      '크롭하','작고','숏','쫄려','작다고','작다','좁아서','작','크롭','크롭된','미니','크롭해',
      '길었으면','컸으면','좁게','짧지' ,'작지','크롭이',
      '숏하','사이즈업하','크롭하','숏한','크롭하긴','크롭이','크롭해','크롭느낌','크롭된','사이즈업',
       '작','숏','짧긴','크롭','크롭한','타이트','크롭이라','작긴',
       '작','짧','좁']

big = ['큰',' 긴','커요','크고','크긴','넉넉하고','길어서','넉넉하게','작다는'
     '넉넉한','클','길고','큰데','크지','크지만','부해','크니깐','크더라구요','컸어도','넓음','큼직하게',
     '박시하네요','넉넉할','헐렁하게','큼직하니','컸음','큽니','긴데','길어','길게','넓어서','넓어','박시하고','컸어요',
     '박시','크다고','크다는','넓게','넓고','커용','길다는','크지도','큼','크다','길었어요','넉넉함','큰데도','덮는데',
     '덮이고','컸고','벙벙해서','크더라고요','컸을','크거나','크더라','컸었는데','컸어용','컸습니다','덮네요',
     '큼지막해서','덥는','박시하게','크네요','클까','넉넉하구','박시하','컸는데','넉넉하네요','크게','크니까','넉넉한데','길어요',
     '컸지만','헐렁한','크네','넉넉해','펑퍼짐한','덮어용','박시하니','헐렁해','넉넉하지''넉넉하구요','커','큽니다','넉넉하니',
     '길지','박시한데','덮어요','길긴','넉넉해요','넓은','덮히는', '넓긴','덮어줘서','덮은','넉넉해서','큼직해서','기네','낙낙',
     '흘러내리는','덮어서','벙벙한','커서','큰것','박시핏','오퍼핏','낙낙해','여유핏','널널함','덮어','넉넉합니다','크면',
     '덮고','가오리','루즈핏','펑퍼짐','길었지만','헐렁하고','남아요','아방핏','박스핏','널널','오바핏','덮습니다','넉넉히',
     '헐렁해서','접어도','접어','접어서','널','넓어요','덮고도','오버','와이드','낭낭','헐렁','접으면','짧았으면','접고',
     '버핏','접어야','잡아먹힌','수선해서','오버핏','오버사이즈','오버사이징해서','낙낙하다','낙낙한데','낙낙해요','오버핏이예요','오버핏입니당',
       '요박시하','컸어용','낙낙합니','어벙벙하','와이드핏','크니깐','크네요옷','널널해요','오버핏나와요','널럴하',
       '요넉넉하','엄청큼','여유핏','오버느낌','아방핏','오버핏은','넓긴','넉넉함','입니다오버핏','큽니다오버핏',
       '큽니다제','큽니당','오버해요','요박시한','헐렁하긴','헐렁거리','아방방한','아방방','박시합니','박시했다',
       '빅사이즈','받았습니다오버핏','빡시하','루즈핏으로','낭낭한','오버핏이긴','박스핏','널널함','어벙벙',
       '요넉넉한','오퍼핏','오버핏이구요','낙낙해','낭낭해서','넉넉하구','넉넉한데','아방해','오버핏이라서',
       '낭낭하','박시해','커용','넉넉해','펑퍼짐','박시하지','널널','좋아요오버핏','어벙벙한','널널하','오바핏',
       '오버하지','오버핏하','박시합니다','오버핏되','박시함','버핏','박시한','낙낙하니','박시해요','크네용',
       '낙낙해서','큼직해서','루즈해','느슨','널널합니다','아방아','아방아방','루즈핏이','박시하','세미오버',
       '세미오버핏','루즈해서','오버핏이라','루즈핏이라','오버핏이네요','오버핏입니','어벙','오버해','박시한데',
       '아방','오버핏이에요','예뻐요오버핏','큼지막','박시핏','오버해서','루즈한','오버핏인데','넉넉하구요','박시할',
       '큼직','오버한','오버핏이','낙낙하','아방하','롱','오버핏으로','낙낙한','오버핏입니다','박시해서','와이드',
       '요오버핏','넉넉해서','큽니','오버','오버하','루즈','루즈핏','박시','넉넉','크긴','아방한','널널한',
       '헐렁','길긴','와이드하','아방해서','세미오버사이즈',             
        '널','덮','넓','벙벙하','덮이','덮히','커다랗','덥히','넉넉해지','뒤집'             
       '넉넉']

In [8]:
total     = sizedict_nodup(total)
chongjang = sizedict_nodup(chongjang)
shoulder  = sizedict_nodup(shoulder)
arm       = sizedict_nodup(arm)
chest     = sizedict_nodup(chest)
small     = sizedict_nodup(small)
big       = sizedict_nodup(big)

중복제거 전:63개
중복제거 후:51개
----------
중복제거 전:25개
중복제거 후:20개
----------
중복제거 전:28개
중복제거 후:19개
----------
중복제거 전:22개
중복제거 후:16개
----------
중복제거 전:11개
중복제거 후:9개
----------
중복제거 전:87개
중복제거 후:74개
----------
중복제거 전:285개
중복제거 후:253개
----------


사이즈, 총장, 어깨, 소매, 가슴에 대한 정보 추출(0/1 추출)

In [9]:
from tqdm import tqdm #진행상황 알려줌

#전체,총장,어깨,소매,가슴 키워드 탐지 >> 크다,작다 존재 여부 확인, 나머지는 보통이다로 간주
def get_keywords(keywords,df,keyword_column_name):
    start = time.time()
    review = df['review']
    df['new_column'] = str('')
    #리뷰 하나를 불러와서 > 키워드 각각이 존재하는지 탐색
    for i in tqdm(range(len(review))):
        keywords_search = []
        for j in keywords:
            if re.findall(j, review[i]):
                a = re.findall(j +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i]) #키워드 +단어
                aa = re.findall(j + ' '+'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review[i])#키워드 +띄고 단어
                b = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+' + j ,review[i]) #단어 + 키워드
                bb = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + j ,review[i])#단어 + 띄고 키워드
                
                #앞에 지만/데/고/보다 등이 들어있으면 키워드에 대한 수식어구가 아님! 
                if(len(b)!=0):
                    if(('지만' in b[0])|('데' in b[0])|('고' in b[0])|('보다' in b[0])|
                       ('도' in b[0])|('서' in b[0])|('요' in b[0])):
                        b=[]
                    else:
                        pass
                else:
                    pass
                if(len(bb)!=0):
                    if(('지만' in bb[0])|('데' in bb[0])|('고' in bb[0])|('보다' in bb[0])|
                       ('도' in bb[0])|('서' in bb[0])|('요' in bb[0])):
                        bb=[]
                    else:
                        pass
                else:
                    pass
                keywords_search.extend(a)
                keywords_search.extend(aa)
                keywords_search.extend(b)
                keywords_search.extend(bb)
        
        #탐색된 키워드가 존재하면 구분자 , 를 기준으로 문자열 생성 후 new_column에 입력
        if len(keywords_search) != 0:
            keywords_o = ','.join(x for x in keywords_search)
            df['new_column'][i] = keywords_o 
        else:
            df['new_column'][i] = '0'
    
    a = df.rename(columns = {'new_column':keyword_column_name})
    end = time.time()
    print("{:.5f} sec".format(end-start))
    return a[keyword_column_name]

In [10]:
keyword_column_name = 'total_keyword'
df_total            = get_keywords(total,df_before_nlp,keyword_column_name)
keyword_column_name = 'chongjang_keyword'
df_chongjang        = get_keywords(chongjang,df_before_nlp,keyword_column_name)
keyword_column_name = 'shoulder_keyword'
df_shoulder         = get_keywords(shoulder,df_before_nlp,keyword_column_name)
keyword_column_name = 'chest_keyword'
df_chest            = get_keywords(chest,df_before_nlp,keyword_column_name)
keyword_column_name = 'arm_keyword'
df_arm              = get_keywords(arm,df_before_nlp,keyword_column_name)

df_keywords         = pd.concat([df_before_nlp,df_total,df_chongjang,df_chest,df_shoulder,df_arm],axis = 1)
df_keywords =df_keywords.drop(['new_column'],axis = 1)
df_keywords_origin = df_keywords.copy()

  0%|                                                                                        | 0/20037 [00:00<?, ?it/s]<ipython-input-9-0f38161e8a30>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_column'][i] = keywords_o
<ipython-input-9-0f38161e8a30>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_column'][i] = '0'
  2%|█▋                                                                          | 447/20037 [00:00<00:10, 1879.91it/s]

8.30908 sec


  3%|█▉                                                                          | 523/20037 [00:00<00:07, 2630.40it/s]

5.26055 sec


  6%|████▏                                                                      | 1125/20037 [00:00<00:03, 5729.06it/s]

5.68205 sec


  1%|▋                                                                           | 195/20037 [00:00<00:11, 1745.33it/s]

3.28359 sec


100%|██████████████████████████████████████████████████████████████████████████| 20037/20037 [00:04<00:00, 4870.25it/s]

4.11983 sec


In [11]:
print('전체:{}, 총장:{},가슴:{},소매:{},어깨{}'.format(df_keywords[df_keywords['total_keyword']!='0'].shape,
                                             df_keywords[df_keywords['chongjang_keyword']!='0'].shape,
                                             df_keywords[df_keywords['chest_keyword']!='0'].shape,
                                             df_keywords[df_keywords['arm_keyword']!='0'].shape,
                                             df_keywords[df_keywords['shoulder_keyword']!='0'].shape))

전체:(12414, 23), 총장:(2142, 23),가슴:(407, 23),소매:(1111, 23),어깨(620, 23)


- Big, Small 감지

In [12]:
#small,big

#big은 '크다' 언어사전, small은 '작다' 언어사전 리스트
def big_small(big,small,df,name_big_small):
    start = time.time()
    review = df[name_big_small+'_keyword']
    df[name_big_small + '_big'] = str('')
    df[name_big_small + '_small'] = str('')
    for i in tqdm(range(len(review))):
        big_search = []
        small_search = []
        
        for j in big:
            if re.findall(j, review[i]):
                a = re.findall(j,review[i]) #키워드
                big_search.extend(a)
               
        for j in small:
            if re.findall(j, review[i]):
                a = re.findall(j,review[i]) #키워드
                small_search.extend(a)
            
        if len(big_search) != 0:
            df[name_big_small + '_big'][i] = 1
        else:
            df[name_big_small + '_big'][i] = 0
        
        if len(small_search) != 0:
            df[name_big_small + '_small'][i] = 1
        else:
            df[name_big_small + '_small'][i] = 0
        #time.sleep(0.2)
    
    end = time.time()
    print("{:.5f} sec".format(end-start))
    return df[[name_big_small + '_big',name_big_small + '_small']]

In [13]:
name_big_small      = 'total'
total_big_small     = big_small(big,small,df_keywords,name_big_small)
name_big_small      = 'chongjang'
chongjang_big_small = big_small(big,small,df_keywords,name_big_small)
name_big_small      = 'arm'
arm_big_small       = big_small(big,small,df_keywords,name_big_small)
name_big_small      = 'chest'
chest_big_small     = big_small(big,small,df_keywords,name_big_small)
name_big_small      = 'shoulder'
shoulder_big_small  = big_small(big,small,df_keywords,name_big_small)

  0%|                                                                                        | 0/20037 [00:00<?, ?it/s]<ipython-input-12-bf470eb3f21a>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_big_small + '_big'][i] = 1
<ipython-input-12-bf470eb3f21a>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name_big_small + '_small'][i] = 0
<ipython-input-12-bf470eb3f21a>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df

35.75356 sec


  0%|▏                                                                             | 40/20037 [00:00<00:50, 393.19it/s]

35.43275 sec


  0%|▏                                                                             | 48/20037 [00:00<00:42, 473.35it/s]

34.66258 sec


  0%|▏                                                                             | 57/20037 [00:00<00:35, 560.31it/s]

35.26711 sec


100%|███████████████████████████████████████████████████████████████████████████| 20037/20037 [00:36<00:00, 554.04it/s]

36.16795 sec


In [14]:
final_df            = df_keywords
final_df_origin     = final_df.copy()

- 관측치별로 big/small이 감지된 최종 데이터프레임 생성 완료

In [16]:
total_both = (final_df['total_big']==1)&(final_df['total_small']==1)
chongjang_both = (final_df['chongjang_big']==1)&(final_df['chongjang_small']==1)
shoulder_both = (final_df['shoulder_big']==1)&(final_df['shoulder_small']==1)
chest_both = (final_df['chest_big']==1)&(final_df['chest_small']==1)
arm_both = (final_df['arm_big']==1)&(final_df['arm_small']==1)
both = (total_both|chongjang_both|shoulder_both|chest_both|arm_both)

print('전체: {}, 총장: {}, 가슴: {}, 소매: {}, 어깨: {}, 11 관측치: {}'.format(total_both.sum(), chongjang_both.sum(), chest_both.sum(),
                                                       arm_both.sum(), shoulder_both.sum(), both.sum()))

전체: 142, 총장: 37, 가슴: 2, 소매: 9, 어깨: 4, 11 관측치: 184


- 나머지 184개 관측치 삭제

In [17]:
real_final_df = final_df.loc[(-both),:].copy()

In [20]:
real_final_df = pd.read_pickle('data/real_final_df_230115.pickle')

모델링 데이터프레임 생성(전체 + 총장, 전체 + 어깨, 전체 + 소매, 전체 + 가슴)

In [21]:
#모델링에 사용할 데이터프레임 생성
df = pd.read_pickle('data/real_final_df_230115.pickle')
dfm = df.loc[:,['gender','height','weight','size','content','총장','어깨너비','가슴단면','소매길이','total_big','total_small',
                 'chongjang_big','chongjang_small','arm_big','arm_small','chest_big','chest_small','shoulder_big','shoulder_small']]
dfm.reset_index(drop=True, inplace=True)
dfm['gender'] = [1 if dfm['gender'][i]=='남성' else 0 for i in range(dfm.shape[0])]

#데이터 타입 변경
col = ['total_big','total_small','chongjang_big','chongjang_small','arm_big','arm_small',
       'chest_big','chest_small','shoulder_big','shoulder_small']
dfm[col] = dfm[col].apply(pd.to_numeric, errors = 'coerce')

In [22]:
#합치기
i=0
for i in range(dfm.shape[0]):
    if(dfm['total_big'][i]==1):
        dfm['chongjang_big'][i] = 1
        dfm['shoulder_big'][i] = 1
        dfm['chest_big'][i] = 1
        dfm['arm_big'][i] = 1
    else:
        pass
    if(dfm['total_small'][i]==1):
        dfm['chongjang_small'][i] = 1
        dfm['shoulder_small'][i] = 1
        dfm['chest_small'][i] = 1
        dfm['arm_small'][i] = 1
        
#둘다 1인거 뽑아내기(몇개인지확인)
print('전체 : {}'.format(dfm[(dfm['total_big']==1)&(dfm['total_small']==1)].shape[0]))
print('어깨 : {}'.format(dfm[(dfm['shoulder_big']==1)&(dfm['shoulder_small']==1)].shape[0]))
print('가슴 : {}'.format(dfm[(dfm['chest_big']==1)&(dfm['chest_small']==1)].shape[0]))
print('소매 : {}'.format(dfm[(dfm['arm_big']==1)&(dfm['arm_small']==1)].shape[0]))
print('총장 : {}'.format(dfm[(dfm['chongjang_big']==1)&(dfm['chongjang_small']==1)].shape[0]))

shoulder  = (dfm['shoulder_big']==1)&(dfm['shoulder_small']==1)
chest     = (dfm['chest_big']==1)&(dfm['chest_small']==1)
arm       = (dfm['arm_big']==1)&(dfm['arm_small']==1)
chongjang = (dfm['chongjang_big']==1)&(dfm['chongjang_small']==1)

shoulder_idx = dfm.loc[shoulder, :]['content'].index
chest_idx    = dfm.loc[chest,    :]['content'].index
arm_idx      = dfm.loc[arm,      :]['content'].index
chongjang_idx= dfm.loc[chongjang,:]['content'].index

<ipython-input-22-9df46db95838>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm['chongjang_big'][i] = 1
<ipython-input-22-9df46db95838>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm['shoulder_big'][i] = 1
<ipython-input-22-9df46db95838>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm['chest_big'][i] = 1
<ipython-input-22-9df46db95838>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

전체 : 0
어깨 : 16
가슴 : 11
소매 : 12
총장 : 66


In [23]:
#11 데이터 전처리
#총장
dfm.loc[chongjang_idx[65],['chongjang_big','chongjang_small']] = 0,0 #19633
dfm.loc[chongjang_idx[64],'chongjang_small'] = 0 #19344
dfm.loc[chongjang_idx[63],['chongjang_big','chongjang_small']] = 0,0 #18989
dfm.loc[chongjang_idx[62],'chongjang_small'] = 0 #18619
dfm.loc[chongjang_idx[61],['chongjang_big','chongjang_small']] = 0,0 #18122
dfm.loc[chongjang_idx[60],['chongjang_big','chongjang_small']] = 0,0 #17490
dfm.loc[chongjang_idx[59],'chongjang_big'] = 0 #17058
dfm.loc[chongjang_idx[58],'chongjang_small'] = 0 #16693
dfm.loc[chongjang_idx[57],'chongjang_big'] = 0 #16063
dfm.loc[chongjang_idx[56],'chongjang_big'] = 0 #15776
dfm.loc[chongjang_idx[55],'chongjang_big'] = 0 #15761
dfm.loc[chongjang_idx[54],'chongjang_big'] = 0 #15583
dfm.loc[chongjang_idx[53],'chongjang_big'] = 0 #15282
dfm.loc[chongjang_idx[52],'chongjang_big'] = 0 #14787
dfm.loc[chongjang_idx[51],'chongjang_big'] = 0 #14326
dfm.loc[chongjang_idx[50],'chongjang_small'] = 0 #14259
dfm.loc[chongjang_idx[49],['chongjang_big','chongjang_small']] = 0,0 #14055
dfm.loc[chongjang_idx[48],['chongjang_big','chongjang_small']] = 0,0 #14051
dfm.loc[chongjang_idx[47],'chongjang_big'] = 0 #13987
dfm.loc[chongjang_idx[46],'chongjang_small'] = 0 #13844
dfm.loc[chongjang_idx[45],['chongjang_big','chongjang_small']] = 0,0 #13819
dfm.loc[chongjang_idx[44],['chongjang_big','chongjang_small']] = 0,0 #13096
dfm.loc[chongjang_idx[43],'chongjang_big'] = 0 #13089
dfm.loc[chongjang_idx[42],'chongjang_big'] = 0 #13055
dfm.loc[chongjang_idx[41],'chongjang_big'] = 0 #12782
dfm.loc[chongjang_idx[40],'chongjang_big'] = 0 #12457
dfm.loc[chongjang_idx[39],'chongjang_big'] = 0 #11711
dfm.loc[chongjang_idx[38],'chongjang_big'] = 0 #11694
dfm.loc[chongjang_idx[37],'chongjang_big'] = 0 #11682
dfm.loc[chongjang_idx[36],'chongjang_big'] = 0 #10991
dfm.loc[chongjang_idx[35],'chongjang_big'] = 0 #100801
dfm.loc[chongjang_idx[34],'chongjang_big'] = 0 #10787
dfm.loc[chongjang_idx[33],'chongjang_big'] = 0 #10650
dfm.loc[chongjang_idx[32],'chongjang_small'] = 0 #10591
dfm.loc[chongjang_idx[31],'chongjang_big'] = 0 #10533
dfm.loc[chongjang_idx[30],'chongjang_big'] = 0 #10516
dfm.loc[chongjang_idx[29],['chongjang_big','chongjang_small']] = 0,0 #10072
dfm.loc[chongjang_idx[28],'chongjang_big'] = 0 #8761
dfm.loc[chongjang_idx[27],'chongjang_big'] = 0 #8687
dfm.loc[chongjang_idx[26],'chongjang_big'] = 0 #8508
dfm.loc[chongjang_idx[25],'chongjang_small'] = 0 #8379
dfm.loc[chongjang_idx[24],'chongjang_small'] = 0 #8268
dfm.loc[chongjang_idx[23],'chongjang_big'] = 0 #7962
dfm.loc[chongjang_idx[22],'chongjang_big'] = 0 #7479
dfm.loc[chongjang_idx[21],'chongjang_big'] = 0 #7457
dfm.loc[chongjang_idx[20],'chongjang_big'] = 0 #7327
dfm.loc[chongjang_idx[19],'chongjang_big'] = 0 #7309
dfm.loc[chongjang_idx[18],'chongjang_big'] = 0 #6906
dfm.loc[chongjang_idx[17],['chongjang_big','chongjang_small']] = 0,0 #6649
dfm.loc[chongjang_idx[16],'chongjang_small'] = 0 #6452
dfm.loc[chongjang_idx[15],'chongjang_big'] = 0 #6404
dfm.loc[chongjang_idx[14],'chongjang_big'] = 0 #6251
dfm.loc[chongjang_idx[13],'chongjang_big'] = 0 #6201
dfm.loc[chongjang_idx[12],'chongjang_big'] = 0 #6200
dfm.loc[chongjang_idx[11],'chongjang_big'] = 0 #5654
dfm.loc[chongjang_idx[10],'chongjang_big'] = 0 #5640
dfm.loc[chongjang_idx[9],'chongjang_big'] = 0 #4885
dfm.loc[chongjang_idx[8],'chongjang_small'] = 0 #4458
dfm.loc[chongjang_idx[7],'chongjang_big'] = 0 #4162
dfm.loc[chongjang_idx[6],'chongjang_big'] = 0 #4100
dfm.loc[chongjang_idx[5],'chongjang_small'] = 0 #4003
dfm.loc[chongjang_idx[4],['chongjang_big','chongjang_small']] = 0,0 #3776
dfm.loc[chongjang_idx[3],'chongjang_big'] = 0 #3341
dfm.loc[chongjang_idx[2],'chongjang_big'] = 0 #2182
dfm.loc[chongjang_idx[1],'chongjang_big'] = 0 #1022
dfm.loc[chongjang_idx[0],'chongjang_big'] = 0 #1019

#소매
dfm.loc[arm_idx[0],['arm_big','arm_small']] = 0 #3776
dfm.loc[arm_idx[1],'arm_small'] = 0 #3973
dfm.loc[arm_idx[2],'arm_small'] = 0 #8379
dfm.loc[arm_idx[3],'arm_small'] = 0 #8647
dfm.loc[arm_idx[4],'arm_small'] = 0 #14334
dfm.loc[arm_idx[5],'arm_small'] = 0 #14787
dfm.loc[arm_idx[6],'arm_small'] = 0 #14822
dfm.loc[arm_idx[7],'arm_small'] = 0 #17135
dfm.loc[arm_idx[8],'arm_small'] = 0 #17267
dfm.loc[arm_idx[9],'arm_small'] = 0 #17490
dfm.loc[arm_idx[10],'arm_small'] = 0 #17582
dfm.loc[arm_idx[11],'arm_small'] = 0 #19344


#가슴
dfm.loc[chest_idx[0],'chest_small'] = 0 #2291
dfm.loc[chest_idx[1],'chest_small'] = 0 #3481
dfm.loc[chest_idx[2],['chest_small','chest_big']] = 0,0 #3891
dfm.loc[chest_idx[3],'chest_small'] = 0 #3973
dfm.loc[chest_idx[4],['chest_small','chest_big']] = 0,0 #5856
dfm.loc[chest_idx[5],['chest_small','chest_big']] = 0,0 #5858
dfm.loc[chest_idx[6],'chest_small'] = 0 #6651
dfm.loc[chest_idx[7],'chest_small'] = 0 #8647
dfm.loc[chest_idx[8],['chest_small','chest_big']] = 0 #14822
dfm.loc[chest_idx[9],'chest_big'] = 0 #15483
dfm.loc[chest_idx[10],['chest_big','chest_small']] = 0,0 #16142


#어깨
dfm.loc[shoulder_idx[0],'shoulder_small'] = 0 #2291
dfm.loc[shoulder_idx[1],'shoulder_small'] = 0 #3481
dfm.loc[shoulder_idx[2],'shoulder_small'] = 0 #3891
dfm.loc[shoulder_idx[3],'shoulder_big']   = 0 #3905
dfm.loc[shoulder_idx[4],'shoulder_small'] = 0 #3973
dfm.loc[shoulder_idx[5],['shoulder_big','shoulder_small']] = 0,0 #5856
dfm.loc[shoulder_idx[6],['shoulder_big','shoulder_small']] = 0,0 #5858
dfm.loc[shoulder_idx[7],'shoulder_small'] = 0 #6651
dfm.loc[shoulder_idx[8],'shoulder_small'] = 0 #8647
dfm.loc[shoulder_idx[9],'shoulder_big'] = 0 #11917
dfm.loc[shoulder_idx[10],['shoulder_big','shoulder_small']] = 0,0 #14822
dfm.loc[shoulder_idx[11],'shoulder_big'] = 0 #15483
dfm.loc[shoulder_idx[12],['shoulder_big','shoulder_small']] = 0,0 #16054
dfm.loc[shoulder_idx[13],['shoulder_big','shoulder_small']] = 0,0 #16142
dfm.loc[shoulder_idx[14],['shoulder_big','shoulder_small']] = 0,0 #17721
dfm.loc[shoulder_idx[15],'shoulder_small'] = 0 #18183

In [24]:
#중복되는거있는지확인
print(dfm[(dfm['total_big']==1)&(dfm['total_small']==1)].shape[0])
print(dfm[(dfm['shoulder_big']==1)&(dfm['shoulder_small']==1)].shape[0])
print(dfm[(dfm['arm_big']==1)&(dfm['arm_small']==1)].shape[0])
print(dfm[(dfm['chest_big']==1)&(dfm['chest_small']==1)].shape[0])
print(dfm[(dfm['chongjang_big']==1)&(dfm['chongjang_small']==1)].shape[0])

0
0
0
0
0


In [25]:
dfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19853 entries, 0 to 19852
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   gender           19853 non-null  int64  
 1   height           19853 non-null  float64
 2   weight           19853 non-null  float64
 3   size             19853 non-null  object 
 4   content          19853 non-null  object 
 5   총장               19853 non-null  float64
 6   어깨너비             19035 non-null  float64
 7   가슴단면             19853 non-null  float64
 8   소매길이             19852 non-null  float64
 9   total_big        19853 non-null  int64  
 10  total_small      19853 non-null  int64  
 11  chongjang_big    19853 non-null  int64  
 12  chongjang_small  19853 non-null  int64  
 13  arm_big          19853 non-null  int64  
 14  arm_small        19853 non-null  int64  
 15  chest_big        19853 non-null  int64  
 16  chest_small      19853 non-null  int64  
 17  shoulder_big

In [35]:
dfm.to_pickle('data/Modeling_DF_230116.pickle')

0    19592
1      261
Name: arm_small, dtype: int64